In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
val=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")
sub=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv")

In [3]:
zero, one =np.bincount(df['Class'])

In [4]:
zero_df=df.loc[df['Class']==0]
one_df=df.loc[df['Class']==1]

In [5]:
zero_df=zero_df.dropna()
zero_df.shape

(446, 58)

In [6]:
one_df=one_df.fillna(df.median())
zero_df=zero_df.fillna(df.median())

/tmp/ipykernel_20/3900790800.py:1: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  one_df=one_df.fillna(df.median())
/tmp/ipykernel_20/3900790800.py:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  zero_df=zero_df.fillna(df.median())


In [7]:
#zero_df=zero_df.drop(zero_df.sample(106).index)#

In [8]:
zero_df.shape

(446, 58)

In [9]:
data=pd.concat([one_df,zero_df])

In [10]:
data=data.sample(frac=1,random_state=42)
data = data.reset_index(drop=True)
#data=df
#data=data.fillna(data.median())

In [11]:
from sklearn.preprocessing import LabelEncoder

data['EJ'] = LabelEncoder().fit_transform(data['EJ'])
data=data.drop(['Id'], axis=1)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [12]:
val['EJ'] = LabelEncoder().fit_transform(val['EJ'])
val=val.drop(['Id'], axis=1)

In [13]:
X=data.drop(["Class"], axis=1)
y=data["Class"]

In [14]:
def d_competition_log_loss(y_true, y_pred):
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    #y_pred = y_pred.T[1]
    extracted_labels = y_pred.get_label()

    y_pred = extracted_labels.flatten()
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    w_0 = (N_0 + N_1) / N_0
    w_1 = (N_0 + N_1) / N_1
    # Calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-5, 1 - 1e-5)
    p_0 = 1 - p_1
    # Calculate the average log loss for each class
    sum_0 = -np.sum((1 - y_true) * np.log(p_0))
    sum_1 = -np.sum(y_true * np.log(p_1))
    to_output = ((w_0 / N_0) * sum_0 + (w_1 / N_1) * sum_1) / (w_0 + w_1)

    return to_output

In [15]:
def competition_log_loss(y_true, y_pred):
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # calculate the weights for each class to balance classes
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    # calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # calculate the summed log loss for each class
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    # calculate the weighted summed logarithmic loss
    balanced_log_loss = (w_0 * log_loss_0 + w_1 * log_loss_1) / 2
    # return the average log loss
    return balanced_log_loss

In [16]:
import xgboost as xgb
import lightgbm as lgbm

In [17]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.15, random_state=1021)

In [18]:
dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)
dmat=xgb.DMatrix(X,label=y)

In [19]:
params_1={'objective':'binary:logistic','n_estimators': 350, 'max_depth': 4, 'learning_rate': 0.0906233089433723, 'subsample': 0.6030705840768126, 'colsample_bytree': 0.5195287910467131, 'gamma': 0.0684625394566348,'disable_default_eval_metric':True}

In [20]:
#ec1_eval_set = [(x_train, y_train), (x_test, y_test)]


In [21]:
#model_1=xgb.XGBClassifier(**params_1)

In [22]:
#model_1.fit(x_train, y_train, early_stopping_rounds = 10,eval_metric=['error','logloss'],          eval_set = ec1_eval_set, verbose = True)

In [23]:
model_1 = xgb.train(params_1, dmat,350,custom_metric=competition_log_loss)

[19:05:01] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.



In [24]:
y_pred1 = model_1.predict(dtest)

In [25]:
y_pred1

array([9.88192856e-01, 4.92153980e-04, 3.12964723e-04, 3.64664174e-03,
       4.95837582e-03, 9.80559364e-03, 1.45348182e-04, 1.71396509e-03,
       1.48030054e-02, 3.14468294e-02, 5.78935957e-03, 1.13319504e-04,
       1.70563173e-04, 3.59258178e-04, 3.64355539e-04, 1.62776969e-02,
       1.87279843e-02, 1.97220929e-02, 3.33720213e-03, 2.50624144e-03,
       1.73107057e-03, 4.07836633e-04, 5.47934975e-03, 1.98043860e-03,
       9.89935219e-01, 9.75467861e-01, 9.78335202e-01, 5.43985632e-04,
       3.35782692e-02, 5.50287776e-02, 1.91795416e-02, 5.66376885e-03,
       2.15736814e-04, 2.49240771e-02, 3.24604986e-03, 5.52793546e-03,
       1.33574670e-02, 1.73661334e-04, 4.58716489e-02, 9.63785529e-01,
       3.71387191e-02, 6.95432886e-04, 9.99022603e-01, 9.88896645e-05,
       9.24486842e-04, 7.69600214e-04, 7.20753521e-03, 1.43031869e-03,
       1.77060987e-03, 4.06291219e-05, 1.29959325e-03, 4.14195191e-03,
       8.44444148e-03, 2.65568146e-03, 9.99465883e-01, 1.38300087e-03,
      

In [26]:
params_2={'objective':'binary:logistic','n_estimators': 450, 'max_depth': 6, 'learning_rate': 0.039932167981249315, 'subsample': 0.555650642024057, 'colsample_bytree': 0.5154209971115191, 'gamma': 0.14496323622911708}

In [27]:
model_2 = xgb.train(params_2, dmat,450,custom_metric=competition_log_loss)

[19:05:02] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.



In [28]:
y_pred2=model_2.predict(dtest)

In [29]:
params_3 = {
    "objective": "binary",
    "n_estimators": 1000,
    "learning_rate": 1.870736190736596,
    "num_leaves": 860,
    "max_depth": 6,
    "lambda_l1": 0,
    "lambda_l2": 15,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
}

In [30]:
lgbm_1=lgbm.LGBMClassifier(**params_3)
lgbm_1.n_estimators=1000


In [31]:
lgbm_1.fit(X,y,eval_metric=competition_log_loss)

[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=15, reg_lambda=0.0 will be ignored. Current value: lambda_l2=15
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, lambda_l1=0, lambda_l2=15,
               learning_rate=1.870736190736596, max_depth=6, n_estimators=1000,
               num_leaves=860, objective='binary')

In [32]:
y_pred3=lgbm_1.predict_proba(x_test)[:,1]

In [33]:
params_4={
    "objective": "binary",
    "n_estimators": 6900,
    "learning_rate": 1.3992536071417663,
    "num_leaves": 200,
    "max_depth": 12,
    "lambda_l1": 0,
    "lambda_l2": 15,
    "bagging_fraction": 0.5,
    "bagging_freq": 1,
}

In [34]:
lgbm_2=lgbm.LGBMClassifier(**params_4)
lgbm_2.n_estimators=6900

In [35]:
lgbm_2.fit(X,y,eval_metric=competition_log_loss)

[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=15, reg_lambda=0.0 will be ignored. Current value: lambda_l2=15
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5


LGBMClassifier(bagging_fraction=0.5, bagging_freq=1, lambda_l1=0, lambda_l2=15,
               learning_rate=1.3992536071417663, max_depth=12,
               n_estimators=6900, num_leaves=200, objective='binary')

In [36]:
y_pred4=lgbm_2.predict_proba(x_test)[:,1]

In [37]:
params_5={'objective':'binary:logistic','n_estimators': 900, 'max_depth': 6, 'learning_rate': 0.025203388152450566, 'subsample': 0.553669304457063, 'colsample_bytree': 0.7294557632232641, 'gamma': 0.0017071844762570072}

In [38]:
model_3 = xgb.train(params_5, dmat,900, custom_metric=competition_log_loss)

[19:05:18] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.



In [39]:
y_pred5 = model_3.predict(dtest)
y_pred5

array([9.7880471e-01, 8.6758414e-04, 6.5787585e-04, 3.4426602e-03,
       5.8749719e-03, 1.0260904e-02, 2.1506668e-04, 2.3779483e-03,
       1.9880621e-02, 3.5556439e-02, 8.5758073e-03, 4.2955452e-04,
       2.2704879e-04, 1.2885579e-03, 1.1537189e-03, 2.5382705e-02,
       2.3959192e-02, 2.1958286e-02, 3.3386757e-03, 2.7661510e-03,
       2.0368397e-03, 6.2710734e-04, 6.2964177e-03, 3.5537498e-03,
       9.8618710e-01, 9.7526813e-01, 9.6328110e-01, 1.4415728e-03,
       4.3616924e-02, 6.7909658e-02, 1.4628992e-02, 6.0038776e-03,
       5.5064860e-04, 3.0951444e-02, 3.9074244e-03, 5.3988774e-03,
       1.8881418e-02, 4.9449387e-04, 3.8966935e-02, 9.4967002e-01,
       3.3021640e-02, 1.6554175e-03, 9.9802446e-01, 4.2157990e-04,
       1.0718191e-03, 1.2497469e-03, 4.8466879e-03, 2.3722539e-03,
       2.5920307e-03, 1.6309883e-04, 1.1807312e-03, 8.2682539e-03,
       1.0242456e-02, 5.0046560e-03, 9.9898940e-01, 4.0631485e-03,
       9.9604321e-01, 9.6740896e-01, 2.3728567e-03, 7.2352728e

In [40]:
y_pred=(y_pred1+y_pred2+y_pred3+y_pred4+y_pred5)/5
y_pred

array([9.88697009e-01, 4.21423790e-04, 4.84507009e-04, 2.18196272e-03,
       3.25746413e-03, 6.70970452e-03, 1.84357904e-04, 1.68583481e-03,
       1.60904249e-02, 2.55601966e-02, 4.75586981e-03, 2.28851650e-04,
       1.84631543e-04, 6.23633290e-04, 7.14243023e-04, 1.53014137e-02,
       1.46070736e-02, 1.37829470e-02, 2.95450090e-03, 2.24137677e-03,
       1.17425329e-03, 3.56042456e-04, 3.62075291e-03, 2.65369905e-03,
       9.89286203e-01, 9.80398421e-01, 9.81331449e-01, 9.01869408e-04,
       2.77675665e-02, 3.90466219e-02, 1.10438566e-02, 3.89043621e-03,
       3.42431697e-04, 2.21685126e-02, 2.53402427e-03, 3.90826728e-03,
       1.16937759e-02, 2.87208377e-04, 2.66526631e-02, 9.67898477e-01,
       2.29731043e-02, 8.31606254e-04, 9.98875354e-01, 2.24929051e-04,
       7.93992805e-04, 7.04546448e-04, 4.17777574e-03, 1.81444643e-03,
       1.94635096e-03, 1.40972453e-04, 1.17612571e-03, 5.80413673e-03,
       5.69533342e-03, 3.09866221e-03, 9.99386139e-01, 2.22707560e-03,
      

In [41]:
from sklearn.metrics import accuracy_score

# Convert probabilities to binary predictions


y_pred_binary = [1 if p >= 0.5 else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 100.00%


In [42]:
from sklearn.metrics import roc_auc_score

auc_score = roc_auc_score(y_test, y_pred)

In [43]:
auc_score

1.0

In [44]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, y_pred_binary)

# Extract true negatives (TN), false positives (FP), false negatives (FN), and true positives (TP) from the confusion matrix
TN, FP, FN, TP = confusion.ravel()

# Calculate the false positive rate (FPR) and false negative rate (FNR)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)

# Print the false positive rate and false negative rate
print("False Positive Rate (FPR):", FPR)
print("False Negative Rate (FNR):", FNR)

False Positive Rate (FPR): 0.0
False Negative Rate (FNR): 0.0


In [45]:
competition_log_loss(y_test,y_pred)

0.011587125946304578

In [46]:
competition_log_loss(y_test,y_pred1)

0.014021043264594683

In [47]:
competition_log_loss(y_test,y_pred2)

0.025952953605155926

In [48]:
competition_log_loss(y_test,y_pred3)

0.00048775504977089664

In [49]:
competition_log_loss(y_test,y_pred4)

0.00016889485473946613

In [50]:
competition_log_loss(y_test,y_pred5)

0.017975901105273806

In [51]:
lb=np.zeros(val.shape[0])
label=val.shape[0]

In [52]:
dval = xgb.DMatrix(val, label=lb)

In [53]:
pred1=model_1.predict(dval)
pred2=model_2.predict(dval)
pred5=model_3.predict(dval)
pred3=lgbm_1.predict_proba(val)[:,1]
pred4=lgbm_2.predict_proba(val)[:,1]
pred=(pred1+pred2+pred3+pred4+pred5)/5

In [54]:
pred

array([0.59107808, 0.59107808, 0.59107808, 0.59107808, 0.59107808])

In [55]:
# Assuming pred is the input array
for i in range(len(pred)):
    if pred[i] > 0.8:
        pred[i] = 0.99999
    if pred[i]<0.2:
        pred[i]=0.00001


In [56]:
pred=pred.astype(float)
pred.resize(label,1)
pred.shape
pred

array([[0.59107808],
       [0.59107808],
       [0.59107808],
       [0.59107808],
       [0.59107808]])

In [57]:
sub[['class_1']]=pred
sub[['class_0']]= 1-pred

In [58]:
sub.to_csv("submission.csv", index=False)

In [59]:
sub

,Id,class_0,class_1
0,00eed32682bb,0.408922,0.591078
1,010ebe33f668,0.408922,0.591078
2,02fa521e1838,0.408922,0.591078
3,040e15f562a2,0.408922,0.591078
4,046e85c7cc7f,0.408922,0.591078
